# IM(G): One time ComCam Image Ingestion and MTAOS Correction

This notebook is used to execute the [LVV-2228 (1.0)] test script during System Spread Integration Tests on Level 3.  
It is part of the plan [LVV-P81] and of the test cylce [LVV-C176].   
Execution steps are separated by horizontal lines.   
Upon completion, save the notebook and its output as a pdf file to be attached to the test execution in JIRA.  

[LVV-T2228 (1.0)]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2229
[LVV-P81]: https://jira.lsstcorp.org/secure/Tests.jspa#/testPlan/LVV-P81
[LVV-C176]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCycle/LVV-C176

In [ ]:
__executed_by__ = ""  # Put your name here
__executed_on__ = ""  # Put the date of execution here

summit = True  # This is used later to define where Butler stores the images

***

## Initial Setup

log onto the summit nublado  
https://summit-lsp.lsst.codes/  
git clone the ts_notebook repo  

There will be a series of procedures to set up, "slew" and track the telescope before we get an image.    
This is similar to test case [LVV-T2189](https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2189).   

***

## Check ComCam Playback Mode

Verify that ComCam can be use the playback option and that the required images are stored in the right place **TBD**.

---
## Load all the needed libraries

Using the setup procedure, get the remotes and the components ready.

This includes simulators as well as real hardware when available (this will depend on when the test is conducted at NCSA or on level 3 or on the telescope):

- pointing  
- mount ( with the CCW)  
- rotator  
- ready M1M3: raise mirror, turn on FB, clear forces. Note that if used at level 3, we need to have M1M3 LUT use mount telemetry  
- ready M2: turn on FB, clear forces. Note that if used at level 3, we need to have M2 LUT use mount telemetry  
- Get cam hex Ready: check config; make sure LUT is on and has valid inputs; make sure hex is at LUT position  
- Get M2 hex (simulator) Ready: check config; make sure LUT is on and has valid inputs; make sure hex is at LUT position  
- Finally, get the MTAOS CSC ready  

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import rubin_jupyter_utils.lab.notebook as nb
nb.utils.get_node()

In [ ]:
import os
import sys
import asyncio
import logging

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

import lsst.daf.butler as dafButler

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel import MTCS, ComCam
from lsst.ts.observatory.control import RotType

In [ ]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [ ]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()

In [ ]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(40)

In [ ]:
await mtcs.start_task

In [ ]:
comcam = ComCam(domain=domain, log=log)

In [ ]:
comcam.set_rem_loglevel(40)

In [ ]:
await comcam.start_task

In [ ]:
await comcam.enable()

---
## Slew and Track


Using the slew procedure, slew the systems to a specific elevation, azimuth and rotator angle.
Verify that the telemetry is generated.

Find a target around az = 120º and el = 60º and rotator angle at PhysicalSky and 1.8º.

At this position, the rotator stays within a couple of degrees of its initial position. This is because the CCW is not running (MTmount in simulation mode).

target -> az = 120$^o$, el = 60$^o$  

In [ ]:
target = await mtcs.find_target(az=120, el=60, mag_limit=8)

print(f"Target: {target}")

Slew to target:

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=1.9)

***
## Take in-focus image

Once the different components are ready (M1M3, M2, rotator and CCW, hexapods) and tracking, take an image using the take_image command in playback mode.  
This second image should be the one that uses the correction calculated with the first slew.

In [ ]:
exp_focus = await comcam.take_object(15)
print(f"Target exposure: {exp_focus}")

----
## Intra Focus Position 

Using the Camera Hexapod, piston ComCam +1mm

In [ ]:
await mtcs.rem.mthexapod_1.cmd_offset.set_start(z=1000.)

----
## Intra Focus Image


While tracking, take an image with ComCam and check that the header is containing the right telemetry

In [ ]:
exp_intra = await comcam.take_object(15)
print(f"Target 1 exposure: {exp_intra}")

---
## Extra Focus Position

Using the Camera Hexapod, piston ComCam to -1mm

In [ ]:
await mtcs.rem.mthexapod_1.cmd_offset.set_start(z=-2000.)

---
## Extra Focus Image

While tracking, take an image with ComCam and check that the header is containing the right telemetry.

In [ ]:
exp_extra = await comcam.take_object(15)
print(f"Target 1 exposure: {exp_extra}")

---
## Go Back to Focus Position

Put the hexapod back to 0mm.

In [ ]:
await mtcs.rem.mthexapod_1.cmd_offset.set_start(z=1000.)

---
## Stop Tracking

If using MTMount Simulator and CCW Following Mode Disabled, stop tracking to prevent the Rotator to hit the limit switches.

In [ ]:
await mtcs.stop_tracking()

---
## Get Zernike Coefficients

Use the MTAOS Wavefront Estimator Pipeline to calculate the required Zernike Coefficients that represent the Wavefront data.

In [ ]:
await mtcs.rem.mtaos.cmd_runWEP.set_start(visitId=exp_intra[0] - 2021111900000, 
                                          extraId=exp_extra[0] - 2021111900000)

---
## Get Corrections

Use the MTAOS Optical Feedback Controller to retrieve the corrections that should be applied to m1m3, m2, camera hexapod, and m2 hexapod.

In [ ]:
await mtcs.rem.mtaos.cmd_runOFC.start(timeout=60.)

---
## Issue the corrections

Issue the corrections found by the MTAOS OFC to m1m3, m2, camera hexapod, and m2 hexapod. 

In [ ]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

***
## Verify ISR Data

Make sure that the Instrument Signature Removal ran on the intra- and extra-focus data and that this data is accessible via Butler.

In [ ]:
if summit:
    butler = dafButler.Butler("/repo/LSSTComCam/")
else:
    butler = dafButler.Butler("/repo/main/")

In [ ]:
exp_intra_id = {'instrument': 'LSSTComCam', 'detector': 0, 'exposure': exp_intra[0]}

raw_intra = butler.get('raw', dataId=exp_intra_id, collections=["LSSTComCam/raw/all"])
print(raw_intra.getMetadata())

print(80 * "-")

isr_intra = butler.get('raw', dataId=exp_intra_id, collections=["LSSTComCam/raw/all"])
print(isr_intra.getMetadata())

***
## Wrap Up and Shut Down

This cell is not currently included as part of the test execution, but included here as needed to shutdown the systems

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtaos"])

In [ ]:
await mtcs.lower_m1m3()

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm1m3"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm2"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mthexapod_1"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mthexapod_2"])

In [ ]:
await mtcs.standby()

In [ ]:
await comcam.standby()